In [1]:
import pandas as pd
import googlemaps
import sys
sys.path.insert(0, "..")
%load_ext autoreload
%autoreload 2
from src.secrets import API_KEY

In [2]:
df = pd.read_csv("../data/ztm_Gdansk.csv", index_col=0)

In [3]:
df.head()

,stopName,stopDesc,zoneId,zoneName,stopLat,stopLon
0,Dąbrowa Centrum,Gdynia Dąbrowa Centrum,5.0,Gdynia,54.47317,18.46509
1,Kameliowa,Gdynia Kameliowa,5.0,Gdynia,54.47018,18.46820
2,Paprykowa,Gdynia Paprykowa,5.0,Gdynia,54.46784,18.46546
3,Szafranowa,Gdynia Szafranowa,5.0,Gdynia,54.46536,18.46020
4,Giełda Towarowa,Gdynia Giełda Towarowa,5.0,Gdynia,54.46224,18.45337


In [13]:
df_zone = df[~df['zoneName'].isnull()].reset_index(drop=True)

In [14]:
df_empty_zone = df[df['zoneName'].isnull()].reset_index(drop=True)

In [52]:
def split_df(start, stop):
    df = df_empty_zone.iloc[start:stop,:].copy()
    df['coord'] = df.apply(lambda x: f"{x['stopLat']}, {x['stopLon']}", axis=1)
    return df.sort_values('coord')

In [81]:
df_0_100 = split_df(0,100).drop('zoneName', axis=1)

In [83]:
df_100_200 = split_df(100,200).drop('zoneName', axis=1)

In [84]:
df_200_300 = split_df(200,300).drop('zoneName', axis=1)

In [85]:
df_300_400 = split_df(300,400).drop('zoneName', axis=1)

In [86]:
df_400_500 = split_df(400,500).drop('zoneName', axis=1)

In [87]:
df_500_end = split_df(500, 550).drop('zoneName', axis=1)

In [60]:
df_zone['coord'] = df_zone.apply(lambda x: f"{x['stopLat']}, {x['stopLon']}", axis=1)

origin = df_zone[df_zone['stopDesc'] == 'Niedźwiednik']['coord'].iloc[0]
origin

'54.38, 18.55937'

In [4]:
from datetime import datetime
time = datetime.now()

In [61]:
from src.distance_matrix_api import get_time_of_travel

In [62]:
result_0_100 = get_time_of_travel(origin, df_0_100['coord'].to_list(), time)

In [90]:
result_100_200 = get_time_of_travel(origin, df_100_200['coord'].to_list(), time)

In [91]:
result_200_300 = get_time_of_travel(origin, df_200_300['coord'].to_list(), time)

In [92]:
result_300_400 = get_time_of_travel(origin, df_300_400['coord'].to_list(), time)

In [93]:
result_400_500 = get_time_of_travel(origin, df_400_500['coord'].to_list(), time)

In [94]:
result_500_end = get_time_of_travel(origin, df_500_end['coord'].to_list(), time)

In [78]:
def get_zone(result):
    adresses = result['destination_addresses']
    city = [place.split(', ')[1] for place in adresses]
    
    zones = []
    for z in city:
        element = z.split(' ')
        if len(element) > 1:
            zones.append(element[1])
        else:
            zones.append(element[0])
    return zones

In [88]:
df_0_100['zoneName'] = get_zone(result_0_100)

In [95]:
df_100_200['zoneName'] = get_zone(result_100_200)

In [96]:
df_200_300['zoneName'] = get_zone(result_200_300)

In [97]:
df_300_400['zoneName'] = get_zone(result_300_400)

In [98]:
df_400_500['zoneName'] = get_zone(result_400_500)

In [99]:
df_500_end['zoneName'] = get_zone(result_500_end)

In [103]:
correct_zones = ['Gdańsk', 'Sopot', "Gdynia"]
df_lists = [df_0_100, df_100_200, df_200_300, df_300_400, df_400_500, df_500_end]
df_zones_checked = pd.DataFrame()

for df in df_lists:
    df_filtered = df[df['zoneName'].isin(correct_zones)]
    df_zones_checked = df_zones_checked.append(df_filtered)

In [104]:
df_zones_checked

,stopName,stopDesc,zoneId,stopLat,stopLon,coord,zoneName
84,NaN,Sikorskiego (N/Ż),NaN,54.33886,18.61888,"54.33886, 18.618879999999997",Gdańsk
92,NaN,Otomińska (N/Ż),NaN,54.34412,18.51058,"54.34412, 18.510579999999997",Gdańsk
0,NaN,Nabrzeże Przemysłowe,NaN,54.37428,18.66562,"54.374280000000006, 18.66562",Gdańsk
98,NaN,Malczewskiego,NaN,54.45035,18.55304,"54.45035, 18.55304",Sopot
96,NaN,Wejherowska,NaN,54.45304,18.55528,"54.45304, 18.55528",Sopot
...,...,...,...,...,...,...,...
508,NaN,Szkoła Morska,NaN,54.52940,18.53770,"54.5294, 18.5377",Gdynia
525,NaN,Chrzanowskiego - Przychodnia,NaN,54.52996,18.54236,"54.529959999999996, 18.542360000000002",Gdynia
504,NaN,Stocznia Gdynia,NaN,54.53219,18.51689,"54.53219, 18.51689",Gdynia
510,NaN,Janka Wiśniewskiego,NaN,54.53263,18.50903,"54.532630000000005, 18.50903",Gdynia


In [108]:
df_all = df_zone.append(df_zones_checked[df_zone.columns.to_list()]).reset_index(drop=True)

In [109]:
df_all

,stopName,stopDesc,zoneId,zoneName,stopLat,stopLon,coord
0,Dąbrowa Centrum,Gdynia Dąbrowa Centrum,5.0,Gdynia,54.47317,18.46509,"54.473169999999996, 18.46509"
1,Kameliowa,Gdynia Kameliowa,5.0,Gdynia,54.47018,18.46820,"54.470180000000006, 18.4682"
2,Paprykowa,Gdynia Paprykowa,5.0,Gdynia,54.46784,18.46546,"54.46784, 18.46546"
3,Szafranowa,Gdynia Szafranowa,5.0,Gdynia,54.46536,18.46020,"54.46536, 18.4602"
4,Giełda Towarowa,Gdynia Giełda Towarowa,5.0,Gdynia,54.46224,18.45337,"54.46224, 18.45337"
...,...,...,...,...,...,...,...
984,NaN,Szkoła Morska,NaN,Gdynia,54.52940,18.53770,"54.5294, 18.5377"
985,NaN,Chrzanowskiego - Przychodnia,NaN,Gdynia,54.52996,18.54236,"54.529959999999996, 18.542360000000002"
986,NaN,Stocznia Gdynia,NaN,Gdynia,54.53219,18.51689,"54.53219, 18.51689"
987,NaN,Janka Wiśniewskiego,NaN,Gdynia,54.53263,18.50903,"54.532630000000005, 18.50903"


In [110]:
df_all.to_csv("../data/stops_3city_checked.csv")